It seems the classifier with updated parameters is already the best we can get from grid search. But, how about we simply do cross validation in it, is the performance will improve?

In [1]:
import os
os.chdir('C:/Users/eddie/Desktop/Prof. Rumi Chunara/alcohol')
import sklearn
print(sklearn.__version__)
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from scipy.stats import uniform
from evaluation.metric import computeAccuracy, computeF1Score, computeAUC
# from sklearn.metrics import roc_auc_score
# from sklearn.grid_search import RandomizedSearchCV, GridSearchCV
from sklearn.cross_validation import KFold

from pipelines.alcohol import AlcoholPipeline


0.17


Prepare the alcohol data

In [50]:
df = pd.read_csv("./data/alcohol_training_instances.csv")
alc_labels = []
# extracting pre-labeled classification
for i in range(0,len(df.labels)):
    alc_labels.append(df.labels[i][12])
alc_labels = list(map(int, alc_labels))
print(alc_labels[:10])
print(len(alc_labels))
# print(type(alc_labels))
alc_labels = np.asarray(alc_labels)
print(type(alc_labels))
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df, alc_labels, test_size=0.33, random_state=42)

[0, 1, 1, 1, 1, 1, 1, 1, 0, 1]
15651
<class 'numpy.ndarray'>


First we test the performance of classifier with updated parameters. The reason why we choose these parameters please check alcoholClassiferLR.ipynb

In [51]:
clf_alc_LR_updateParams = AlcoholPipeline(global_features=["text"]).pipeline(LogisticRegression())
params_LR = {'features__text__tfidf__dtype':np.int64, 'features__text__tfidf__max_features':71233, 'features__text__tfidf__ngram_range':(2,5) 
            }
# to see all parameters in model
# clf_alc_LR_updateParams.get_params().keys()
clf_alc_LR_updateParams.set_params(**params_LR)

Pipeline(steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text', Pipeline(steps=[('getter', ItemGetter(key='text')), ('tfidf', TfidfVectorizer(analyzer='char', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=T...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [52]:
print('Please make sure the following class equals pandas.core.frame.DataFrame: ')
print(type(X_train))
print('length of training dataset: ', len(X_train))
# print(X_train.index[1:3])
kf = KFold(len(X_train), n_folds=10, shuffle=False, random_state=None)
# print(kf)
# print(type(y_train))


for train_index, test_index in kf:
    print("TRAIN:", train_index, "TEST:", test_index)
#     print('length of training dataset: ', len(train_index))
#     print('length of testing dataset: ', len(test_index))
    X_train_sub = X_train.iloc[train_index]
    y_train_sub = y_train[train_index]
    X_test_sub = X_train.iloc[test_index]
    y_test_sub = y_train[test_index]
    
#     print(type(X_train_sub))
#     print(len(X_train_sub))

    clf_alc_LR_updateParams.fit(X_train_sub, y_train_sub)
    
    # predict in testing set.
    y_alc_LR_updateParams = clf_alc_LR_updateParams.predict(X_test_sub)
    # print('length of testing set: ', len(y_alc_initial))
    computeAccuracy(y_alc_LR_updateParams, y_test_sub)
    auc_score_LR_updateParams = roc_auc_score(y_test_sub,y_alc_LR_updateParams)
    f1_score_LR_updateParams = computeF1Score(y_test_sub, y_alc_LR_updateParams)
    print('F1 Score: ', f1_score_LR_updateParams)
    auc_score_LR_updateParams = computeAUC(clf_alc_LR_updateParams, X_test_sub, y_test_sub, plot=False)
    # auc_score = computeAUC(clf_alc_LR_updateParams, X_train, y_train, X_test, y_test, plot=True, plotTitle="Alcohol ROC Curve")
    print('alcohol LR AUC: ', auc_score_LR_updateParams)

print('----------------------------- after K-fold cross validation -----------------------------')

# predict in testing set.
y_alc_LR_updateParams = clf_alc_LR_updateParams.predict(X_test)
# print('length of testing set: ', len(y_alc_initial))
computeAccuracy(y_alc_LR_updateParams, y_test)
auc_score_LR_updateParams = roc_auc_score(y_test,y_alc_LR_updateParams)
f1_score_LR_updateParams = computeF1Score(y_test, y_alc_LR_updateParams)
print('F1 Score: ', f1_score_LR_updateParams)
auc_score_LR_updateParams = computeAUC(clf_alc_LR_updateParams, X_test, y_test, plot=False)
# auc_score = computeAUC(clf_alc_LR_updateParams, X_train, y_train, X_test, y_test, plot=True, plotTitle="Alcohol ROC Curve")
print('alcohol LR AUC: ', auc_score_LR_updateParams)

Please make sure the following class equals pandas.core.frame.DataFrame: 
<class 'pandas.core.frame.DataFrame'>
length of training dataset:  10486
TRAIN: [ 1049  1050  1051 ... 10483 10484 10485] TEST: [   0    1    2 ... 1046 1047 1048]


D:\Anaconda3\envs\py36\lib\site-packages\sklearn\feature_extraction\text.py:1015: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


length of total comps:  1049
pretrained alcohol classifer:  138 different item in whole dataset
accuracy: 86.844614%
F1 Score:  0.9071332436069987
alcohol LR AUC:  0.9045177383592018
TRAIN: [    0     1     2 ... 10483 10484 10485] TEST: [1049 1050 1051 ... 2095 2096 2097]
length of total comps:  1049
pretrained alcohol classifer:  138 different item in whole dataset
accuracy: 86.844614%
F1 Score:  0.9054794520547946
alcohol LR AUC:  0.8986342846099337
TRAIN: [    0     1     2 ... 10483 10484 10485] TEST: [2098 2099 2100 ... 3144 3145 3146]
length of total comps:  1049
pretrained alcohol classifer:  151 different item in whole dataset
accuracy: 85.605338%
F1 Score:  0.8998009289980092
alcohol LR AUC:  0.8890790012279982
TRAIN: [    0     1     2 ... 10483 10484 10485] TEST: [3147 3148 3149 ... 4193 4194 4195]
length of total comps:  1049
pretrained alcohol classifer:  171 different item in whole dataset
accuracy: 83.698761%
F1 Score:  0.8835942818243703
alcohol LR AUC:  0.890630707898

Now we compare the performance with model without cross validation

In [53]:
clf_alc_LR_updateParams2 = AlcoholPipeline(global_features=["text"]).pipeline(LogisticRegression())
params_LR = {'features__text__tfidf__dtype':np.int64, 'features__text__tfidf__max_features':71233, 'features__text__tfidf__ngram_range':(2,5) 
            }
# to see all parameters in model
# clf_alc_LR_updateParams2.get_params().keys()
clf_alc_LR_updateParams2.set_params(**params_LR)

Pipeline(steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text', Pipeline(steps=[('getter', ItemGetter(key='text')), ('tfidf', TfidfVectorizer(analyzer='char', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=T...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [54]:
clf_alc_LR_updateParams2.fit(X_train, y_train)
# predict in testing set.
y_alc_LR_updateParams = clf_alc_LR_updateParams2.predict(X_test)
# print('length of testing set: ', len(y_alc_initial))
computeAccuracy(y_alc_LR_updateParams, y_test)
auc_score_LR_updateParams = roc_auc_score(y_test,y_alc_LR_updateParams)
f1_score_LR_updateParams = computeF1Score(y_test, y_alc_LR_updateParams)
print('F1 Score: ', f1_score_LR_updateParams)
auc_score_LR_updateParams = computeAUC(clf_alc_LR_updateParams2, X_test, y_test, plot=False)
# auc_score = computeAUC(clf_alc_LR_updateParams2, X_train, y_train, X_test, y_test, plot=True, plotTitle="Alcohol ROC Curve")
print('alcohol LR AUC: ', auc_score_LR_updateParams)

D:\Anaconda3\envs\py36\lib\site-packages\sklearn\feature_extraction\text.py:1015: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


length of total comps:  5165
pretrained alcohol classifer:  750 different item in whole dataset
accuracy: 85.479187%
F1 Score:  0.8954558126568163
alcohol LR AUC:  0.88876528818431


We can see cross validation has little effect on the performance. The fit function within pipeline is robust.